# Zillow Data via Quandl

In [1]:
import os
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import sys
# sys.path.append('C:Users/Rob/repos/labspt15-cityspire-g-ds')
# sys.path

path = r'C:\Users\Rob\repos\labspt15-cityspire-g-ds\\'

In [2]:
# Example query
quandl_url = "rfzykh-NDsw72dmc4Ms4"
quandl.ApiConfig.api_key = quandl_url
query = quandl.get_table('ZILLOW/DATA', indicator_id='ZSFH', region_id='99999')
query

,indicator_id,region_id,date,value
None,,,,
0,ZSFH,99999,2021-02-28,396263.0
1,ZSFH,99999,2021-01-31,389595.0
2,ZSFH,99999,2020-12-31,383270.0
3,ZSFH,99999,2020-11-30,378566.0
4,ZSFH,99999,2020-10-31,375825.0
...,...,...,...,...
189,ZSFH,99999,2005-05-31,169868.0
190,ZSFH,99999,2005-04-30,167105.0
191,ZSFH,99999,2005-03-31,165721.0


(194, 4)

## Data

In [9]:
data = pd.read_csv(path + "ZILLOW_DATA.csv")
data.head()

,indicator_id,region_id,date,value
0,ZATT,3101,1998-01-31,338849.0
1,ZATT,3101,1998-02-28,342993.0
2,ZATT,3101,1998-03-31,346763.0
3,ZATT,3101,1998-04-30,349356.0
4,ZATT,3101,1998-05-31,351981.0


In [10]:
len(data) #125.5 million lines!! 

125522443

In [13]:
data['indicator_id'].unique()

array(['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR',
       'Z1BR', 'Z4BR', 'ISAW', 'SSAW', 'IRAM', 'CRSM', 'CSSM', 'MSAM',
       'IRAW', 'MRAM', 'SSSW', 'LSSM', 'CSAW', 'ISSM', 'SASW', 'MRAW',
       'LRAM', 'ISSW', 'MSAW', 'LSAM', 'CRAM', 'ISAM', 'IRSM', 'SRSW',
       'LSAW', 'NRAW', 'NSAW', 'CRSW', 'SSAM', 'IRSW', 'NRAM', 'CRAW',
       'LRSM', 'SASM', 'SSSM', 'SAAM', 'LRAW', 'CSAM', 'SRAW', 'CSSW',
       'LRSW', 'SRSM', 'SAAW', 'LSSW', 'SRAM', 'NSAM', 'RSNA', 'RSSA'],
      dtype=object)

## Indicators

In [3]:
indicators = pd.read_csv(path + 'ZILLOW_INDICATORS.csv')
indicators.head()

,indicator_id,indicator,category
0,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
1,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
2,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
3,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values


In [4]:
indicators['indicator_id'].unique()

array(['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR',
       'Z1BR', 'LSAW', 'NRAW', 'NSAW', 'CRSW', 'SSAM', 'IRSW', 'NRAM',
       'CRAW', 'LRSM', 'SASM', 'SSSM', 'SAAM', 'LRAW', 'CSAM', 'SRAW',
       'CSSW', 'LRSW', 'SRSM', 'SAAW', 'LSSW', 'SRAM', 'NSAM', 'RSNA',
       'RSSA', 'Z4BR', 'ISAW', 'SSAW', 'IRAM', 'CRSM', 'CSSM', 'MSAM',
       'IRAW', 'MRAM', 'SSSW', 'LSSM', 'CSAW', 'ISSM', 'SASW', 'MRAW',
       'LRAM', 'ISSW', 'MSAW', 'LSAM', 'CRAM', 'ISAM', 'IRSM', 'SRSW'],
      dtype=object)

In [5]:
indicators

,indicator_id,indicator,category
0,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
1,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
2,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
3,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z1BR,ZHVI 1-Bedroom Time Series ($),Home values
9,LSAW,"Median List Price (Smooth, All Homes, Weekly V...",Inventory and sales


## Regions

In [6]:
regions = pd.read_csv(path + 'ZILLOW_REGIONS.csv')
regions.head()

,region_id,region_type,region
0,9,state,California; CA
1,14,state,Florida; FL
2,21,state,Illinois; IL
3,47,state,Pennsylvania; PA
4,394913,metro,"New York, NY"


In [7]:
len(regions)

78730

In [8]:
regions['region_type'].unique()

array(['state', 'metro', 'zip', 'neigh', 'county', 'city'], dtype=object)

# Aggregating Data

In [15]:
# Checking if data has all the same sections
sorted(data['indicator_id'].unique()) == sorted(indicators['indicator_id'].unique())

True